### Association Rules

In [1]:
!pip install apyori

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=6015 sha256=512f5ec698d3523b26db7fa67ea5d87b0d103579f9af15cfe932dbd06c2ae89e
  Stored in directory: a:\wpsystem\s-1-5-21-910491398-3542826876-2768250131-1001\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\77\3d\a6\d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori


In [7]:
import pandas as pd
from apyori import apriori
import numpy as np
# Load dataset
df = pd.read_csv('Groceries_dataset.csv', header=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [ ]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [12]:
#checking for missing values
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [ ]:
#clearing column names
df.columns = df.columns.str.strip()
#altering date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-02-01,whole milk


In [16]:
#integrating transactions per date
df['TransactionID'] = df['Member_number'].astype(str) + '_' + df['Date'].dt.strftime('%Y-%m-%d')
df.head()

,Member_number,Date,itemDescription,TransactionID
0,1808,2015-07-21,tropical fruit,1808_2015-07-21
1,2552,2015-01-05,whole milk,2552_2015-01-05
2,2300,2015-09-19,pip fruit,2300_2015-09-19
3,1187,2015-12-12,other vegetables,1187_2015-12-12
4,3037,2015-02-01,whole milk,3037_2015-02-01


In [17]:
#reformatting dataset to have transactions in one row
basket_df = (
    df.groupby('TransactionID')['itemDescription']
      .apply(lambda x: ','.join(sorted(x.unique())))
      .reset_index()
)
basket_df.head()


,TransactionID,itemDescription
0,1000_2014-06-24,"pastry,salty snack,whole milk"
1,1000_2015-03-15,"sausage,semi-finished bread,whole milk,yogurt"
2,1000_2015-05-27,"pickled vegetables,soda"
3,1000_2015-07-24,"canned beer,misc. beverages"
4,1000_2015-11-25,"hygiene articles,sausage"


In [19]:
#one-hot encoding the dataset
basket_hot = (
    basket_df['itemDescription'].str.get_dummies(',')
)
basket_hot.head()


,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#saving preprocessed dataset to csv
basket_df.to_csv('preprocessed_transactions.csv', index=False)
#saving one-hot encoded dataset to csv
basket_hot.to_csv('onehot_transactions.csv', index=False)

Now we have preprocessed the data into one-hot encoded dataset, which is suitable for association mining. 